
# Intel® Movidius™ Neural Compute Stick (NCS)

This lab shows how the Intel® Distribution of OpenVINO™ toolkit provides hardware abstraction to run the sample object detection application which was built in previous modules on Intel® Movidius™ Neural Compute Stick.


## Connect Intel® Movidius™ Neural Compute Stick to your development laptop

<img src="Movidius.png">


## System check


First make sure the USB rules are set up.

In [ ]:
%%writefile myriad.sh

cat <<EOF > 97-myriad-usbboot.rules
SUBSYSTEM=="usb", ATTRS{idProduct}=="2150", ATTRS{idVendor}=="03e7", GROUP="users", MODE="0666", ENV{ID_MM_DEVICE_IGNORE}="1"
SUBSYSTEM=="usb", ATTRS{idProduct}=="2485", ATTRS{idVendor}=="03e7", GROUP="users", MODE="0666", ENV{ID_MM_DEVICE_IGNORE}="1"
SUBSYSTEM=="usb", ATTRS{idProduct}=="f63b", ATTRS{idVendor}=="03e7", GROUP="users", MODE="0666", ENV{ID_MM_DEVICE_IGNORE}="1"
EOF

In [ ]:
%%bash
sh myriad.sh

- *Note: Replace "xxxxx" with your system-password.*

In [ ]:
!echo "xxxxx" | sudo -S cp 97-myriad-usbboot.rules /etc/udev/rules.d/ >/dev/null 2>&1

In [ ]:
!echo "xxxxx" | sudo -S udevadm control --reload-rules >/dev/null 2>&1

In [ ]:
!echo "xxxxx" | sudo -S udevadm trigger >/dev/null 2>&1


    
Then check if the device is visible with lsusb.

In [ ]:
!lsusb



The output will be If using NCS1,

```
Bus 002 Device 001: ID 1d6b:0003 Linux Foundation 3.0 root hub
Bus 001 Device 015: ID 03e7:2150 
```



If useing NCS2,
```
Bus 002 Device 001: ID 1d6b:0003 Linux Foundation 3.0 root hub
Bus 001 Device 015: ID 03e7:2485 
```


Here ID 03e7:2150 without a description string is the Movidius device.


Setting up the Environment variables

In [ ]:
! /opt/intel/openvino/bin/setupvars.sh


    
## Run the sample application on Intel® Movidius™ Neural Compute Stick (NCS)

Set target hardware as Intel® Movidius™ NCS with -d MYRIAD

In [ ]:
! python3 tutorial1.py -i /opt/intel/workshop/smart-video-workshop/object-detection/Python/Cars\ -\ 1900.mp4 -m /opt/intel/workshop/smart-video-workshop/object-detection/mobilenet-ssd/FP32/mobilenet-ssd.xml -d MYRIAD



You will get an error as Intel® Movidius™ NCS supports only FP16 format. 

<img src="FP32.png">


The Model Optimizer by default generate FP32 IR files if the data type is not particularly specified.

Let's run the Model Optimizer to get IR files in FP16 format suitable for the Intel® Movidius™ NCS by setting the data_type flag to FP16.

In [ ]:
!cd /opt/intel/workshop/smart-video-workshop/object-detection/mobilenet-ssd && mkdir -p FP16

In [ ]:
!cd /opt/intel/openvino/deployment_tools/model_optimizer && python3 mo_caffe.py --input_model /opt/intel/openvino/deployment_tools/tools/model_downloader/object_detection/common/mobilenet-ssd/caffe/mobilenet-ssd.caffemodel -o /opt/intel/workshop/smart-video-workshop/object-detection/mobilenet-ssd/FP16 --scale 256 --mean_values [127,127,127] --data_type FP16


    
Check if the .xml and .bin files are created in folder $SV/object-detection/mobilenet-ssd/FP16.

In [ ]:
 !cd /opt/intel/workshop/smart-video-workshop/object-detection/mobilenet-ssd/FP16 && ls


    
Now run the example application with these new IR files.

In [ ]:
! python3 tutorial1.py -i /opt/intel/workshop/smart-video-workshop/object-detection/Python/Cars\ -\ 1900.mp4 -m /opt/intel/workshop/smart-video-workshop/object-detection/mobilenet-ssd/FP16/mobilenet-ssd.xml -d MYRIAD